<a href="https://colab.research.google.com/github/sadnow/AnimationKit-AI_Upscaling-Interpolation_RIFE-RealESRGAN/blob/testing/TESTING_AnimationKit_Rife_RealESRGAN_Upscaling_Interpolation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AnimationKit AI - Upscaling & Interpolation using Real-ESRGAN+RIFE

**ALPHA 2: Released 9/4/21**

New features:

- Deflickering (P3)
- Upscaling from individual frames (P2)
- Target length (in seconds) for RIFE interpolation

**-WARNING: THIS IS THE TESTING BRANCH! Try the main branch for stability-**

Main branch can be found here: https://github.com/sadnow/AnimationKit-AI_Upscaling-Interpolation_RIFE-RealESRGAN


Credits: Motion smoothing conceived from "Zoom animation processing and motion interpolation" added by https://twitter.com/unltd_dream_co. This part of the script uses [RIFE real-time video interpolation](https://github.com/hzwer/arXiv2020-RIFE) to smooth out the resulting video. 

Upscaling uses Real-ESRGAN (https://github.com/xinntao/Real-ESRGAN). A demo notebook for static images can be found here: https://colab.research.google.com/drive/1k2Zod6kSHEvraybHl50Lys0LerhyTMCo?usp=sharing. The demo was based on the following paper: [''Real-ESRGAN: Training Real-World Blind Super-Resolution with Pure Synthetic Data''](https://arxiv.org/abs/2107.10833). A demo can be found at https://raw.githubusercontent.com/xinntao/Real-ESRGAN

Special thanks to @sportsracer48 and everyone on his Discord. If you want closed beta access to the best VQGAN animation notebook on the planet, check out https://www.patreon.com/sportsracer48

---

Testing notes:

Many ffmpeg examples at https://docs.google.com/document/d/12X_2YoCnPPN7B3OsgX39aYyRF8OF-TVStkFTkKhWrx4/edit

Python modules in Colab: https://medium.com/analytics-vidhya/importing-your-own-python-module-or-python-file-into-colab-3e365f0a35ec

Colab Markdown guide: https://colab.research.google.com/notebooks/markdown_guide.ipynb?hl=es-ni#scrollTo=Lhfnlq1Surtk



In [ ]:
# @title AnimationKit AI is licensed under the MIT License
# @markdown Practical-RIFE, Real-ESRGAN, and any other external tools belong to their respective authors and may vary in licensing agreements.

# Copyright (c) 2021 Sadnow

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

In [ ]:
!nvidia-smi

# Mount Google Drive, Install Libraries, Load Functions

Most of the code is in here. If it looks weird, it's because half-way through development I decided to make it a Python module. It's not quite portable yet, but it's getting close!

In [ ]:
import os, sys, re
from multiprocessing import Process

def need_dir(a):
  import os.path
  from os import path
  if not path.exists(a):
    print("Creating"+a+"...")
    !mkdir -p $a

def need_file(a):
  import os.path
  from os import path
  if not path.isfile(a):
    print("Creating"+a+"...")
    !mkdir -p $a

def check_dir(a):
  import os.path
  from os import path
  if not path.exists(a):
    print("ERROR: "+a+" does not exist!")

def check_file(a):
  import os.path
  from os import path
  if not path.isfile(a):
    print("ERROR: "+a+" does not exist!")


class anKit:
  #def __init__(self):
  #  pass
  import os.path
  from os import path
  #video tools
  def sortFrames(sourceframes, destframes):  #takes frames from input folder, moves to init_frame_storage
    check_dir(sourceframes)
    need_dir(destframes)
    %cd $sourceframes
    print("Copying frames to "+destframes+" for processing...")
    !find -maxdepth 1 -name '*.png' -print0 | xargs -0 cp -t $destframes
    %cd $destframes
    #!find -maxdepth 1 -name '*.png' -print0 | xargs -0 'mv "$0" "${0##*_}"' 
    #!find . -type f -name "*_*.png" -execdir bash -c 'mv "$0" "${0##*_}"' {} \;  #removes anything not numbers
    !find . -type f -name "*.png" -execdir bash -c 'mv "$0" "${0##*_}"' {} \;  #removes anything not numbers
    #print("Padding filenames in "+destframes+".")
    !rename 's/\d+/sprintf("%05d",$&)/e' *  #adds padding to numbers
    print("Finished copying frames to "+destframes+".")
  def splitFrames(sourcefile, destframes):
    check_file(sourcefile)
    need_dir(destframes)
    !ffmpeg -y -r 1 -i $sourcefile -r 1 $destframes/frame%05d.png
  def frames2video(sourceframes,fps,outputmp4):
    check_dir(sourceframes)
    %cd $sourceframes
    #!ffmpeg -r $fps -i '%d_out.png' $outputmp4
    !ffmpeg -y -f image2 -pattern_type glob -i '*.png' $outputmp4
  def runUpscale(esrgan,mpath,scale,input,output):
    check_dir(input)
    need_dir(output)
    %cd $esrgan
    #print(mpath)
    !python inference_realesrgan.py --model_path $mpath --netscale $scale --input $input --output $output
    %cd output
    !find . -type f -name "*_*.png" -execdir bash -c 'mv "$0" "${0##*_}"' {} \;  #removes anything not numbers from filename - needed for rife-frame
  def exportVideo(input,effects,quality,output):  #needs portable
    !ffmpeg -y -i $input $visual_effects -c:v hevc_nvenc -rc vbr -cq $constant_quality -qmin $constant_quality -qmax $constant_quality -b:v 0 $output_path_mp4
  def RIFE_video(fps,exp,input,scale,output):
    %cd /content/Practical-RIFE
    !python3 /content/Practical-RIFE/inference_video.py --fps=$fps --exp=$exp --video=$input --scale $scale --output=$output
  def RIFE_frames(fps,exp,input,scale,output):  #not currently working right
    %cd /content/Practical-RIFE
    input = input + '/'
    !python3 /content/Practical-RIFE/inference_video.py --fps=$fps --exp=$exp --img=$input --scale $scale --output=$output
  def detect_fps(input): #needs portable
    import re
    fps_ffprobe = !ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=avg_frame_rate $input
    fps_unfinished = [str(i) for i in fps_ffprobe] # Converting integers into strings
    fps_unfinishedTwo = str("".join(fps_unfinished)) # Join the string values into one string
    numbers = re.findall('[0-9]+', fps_unfinishedTwo)
    newNum = numbers[0:1]
    strings = [str(integer) for integer in newNum]
    a_string = "".join(strings)
    fps = int(a_string)
    #print("Detected FPS is",fps)
    return fps
  def detect_duration(input):  #needs portable
    import re
    duration_ffprobe = !ffprobe -v error -select_streams v:0 -show_entries stream=duration -of default=noprint_wrappers=1:nokey=1 $input
    duration_unfinished = [str(i) for i in duration_ffprobe] # Converting integers into strings
    duration_unfinishedTwo = str("".join(duration_unfinished)) # Join the string values into one string
    numbers = re.findall('[0-9]+', duration_unfinishedTwo)
    newNum = numbers[0:1]
    strings = [str(integer) for integer in newNum]
    a_string = "".join(strings)
    duration = int(a_string)
    #print("Detected duration INTEGER (in seconds) is",duration)
    return duration
  def exp_calc(): #needs portable
    import numpy as np
    a = measured_fps * measured_duration
    b = target_fps * target_length_seconds
    c = b / a
    l = np.log(c) / np.log(2)
    print("Un-rounded --exp is",l)
    x = round(l)
    if x < 1:
      x = 1
    print("Rounding up to an --exp of ",x)
    return x
  #installation for Colab (untested on other platforms)
  #
  #
  def install_Colab():
    print("Installing libraries for Real-ESRGAN upscaling.")
    !git clone https://github.com/xinntao/Real-ESRGAN.git
    %cd Real-ESRGAN
    !pip -q install basicsr
    !pip -q install facexlib
    !pip -q install gfpgan
    !pip -q install -r requirements.txt
    !python setup.py develop
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth -P experiments/pretrained_models
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth -P experiments/pretrained_models
    print("Finished Installing libraries for Real-ESRGAN upscaling.")
    #
    %cd /content/
    print("Installing libraries for RIFE motion smoothing.")
    !git clone https://github.com/hzwer/Practical-RIFE Practical-RIFE
    !gdown --id 1O5KfS3KzZCY3imeCr2LCsntLhutKuAqj
    #%cd /content/Practical-RIFE
    !7z e RIFE_trained_model_v3.8.zip
    #!7z e /content/Practical-RIFE/RIFE_trained_model_v3.8.zip
    #!7z e /content/RIFE_trained_model_v3.8.zip
    !mkdir /content/Practical-RIFE/train_log
    !mv *.py /content/Practical-RIFE/train_log/
    !mv *.pkl /content/Practical-RIFE/train_log/
    %cd /content/Practical-RIFE/
    !gdown --id 1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc #useless - mp4 demo
    !pip3 install -r requirements.txt
    print("Finsihed Installing libraries for RIFE motion smoothing.")
    #  
  %cd /content/
  def depcrecated_installOldRIFE():
    print("Installing libraries for RIFE motion smoothing.")
    !git clone https://github.com/hzwer/arXiv2020-RIFE RIFE
    !gdown --id 1wsQIhHZ3Eg4_AfCXItFKqqyDMB4NS0Yd
    !7z e RIFE_trained_model_HDv2.zip
    !mkdir /content/RIFE/train_log
    !mv *.pkl /content/RIFE/train_log/
    %cd /content/RIFE/
    !gdown --id 1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc
    !pip3 install -r requirements.txt
    print("Done.")
    print("Finsihed Installing libraries for RIFE motion smoothing.")
  %cd /content/
    
###


#


#



###############################################################

#
#If RIFE (P2) stops for no reason, go to Runtime>Change Runtime type and set the notebook to "high memory"

#Params
mount_google_drive = True #@param {type:"boolean"}

#Mount Google Drive
if mount_google_drive:
  from google.colab import drive
  drive.mount('/content/drive')

#install dependencies
anKit.install_Colab()

# Real-ESRGAN Video Upscaling
Planned additions: Better flickering, faces option, tiles option

---

# Motion interpolation (Practical-RIFE Smoothing)

New: Type in the number of seconds you want your clip to be!

Lower the `target_scale_RIFE` to speed up RIFE (at the cost of resolution). A value of `0.5` would make a 4k video 2k.

*Tip: Save as a high FPS (eg 999) if you plan on doing heavy editing (that way you don't need to re-process at the end.)*

---

# P3: Deflickering & x265 Compression (Experimental, Very Fast)

Sort of like Handbrake - good for large filesizes. Can turn 500mb files into 200mb files with negligable quality loss. Deflickering is good for deflickering stopmotions, VQGAN_CLIP animations, etc.

Also, too high of compressions might require a decent local GPU to view. Will add h264 toggle in the future~

In [15]:
#@markdown # AnimationKit AI
#@markdown >*I aim for this to include *
#@markdown ---
#@markdown ##Input settings
##@markdown >For importing mp4 files:
import_mp4 = False #@param {type:"boolean"}
imported_mp4='/content/drive/MyDrive/pytti_test/videos/uncle.mp4' #@param {type:"string"}
##@markdown `
##@markdown >For importing individual frames:
import_frame_folder='/content/drive/MyDrive/pytti_test/images_out/basingse' #@param {type:"string"}
#@markdown Note: *`import_frame_folder` will be ignored if `import_mp4` is checked.*



#@markdown ---
#@markdown ##Output settings
output_path_mp4='/content/drive/MyDrive/pytti_test/videos/basingse_anKit.mp4' #@param {type:"string"}
target_fps=120#@param {type:"integer"}
target_length_seconds=60#@param {type:"integer"}
model_path='/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus.pth' #@param ['/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus_anime_6B.pth','/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus.pth','/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x2plus.pth'] {type:"string"}
constant_quality=25#@param {type:"slider", min:20, max:50, step:1}
face_enhance_gfpgan=True #@param{type:"boolean"}
#@markdown Default `constant_quality` is `27`, which scrunched a 500mb 40 min video into ~200mb. Higher values = lower filesize, lower quality

#@markdown ---



#@markdown ##Speed Optimization
#input_path='/content/drive/MyDrive/pytti_test/videos/zelda4.mp4' #@param {type:"string"}
target_scale_RIFE='2.0'#@param ['0.25','0.5','1.0','2.0','4.0']{type:"string"}
half_precision_realesrgan=False #@param{type:"boolean"}
#length_multiplier=3#@param {type:"integer"}
#@markdown *These options can speed up processing at the costs of quality.*

#@markdown If you're seeing weird warping in your outputs, try increasing `target_scale_RIFE`.
#@markdown 

#@markdown ---
#@markdown #Visual Effects & Compression
#compress_path='/content/drive/MyDrive/pytti_test/videos/zelda4_16X_180fps.mp4' #@param {type:"string"}
#outputStr = '_tblend2.mp4' #@param {type:"string"}
enable_visual_effects = False #@param {type:"boolean"}
#@markdown Disable visual_effects if you don't have any visual effects checked-
deflicker_on = False #@param {type:"boolean"}
deflicker_avg_frames=76#@param {type:"slider", min:2, max:129, step:1}
#mpdecimate_on = False #@param {type:"boolean"}
#minterpolate_on = False #@param {type:"boolean"}
#minterpolate_fps=60#@param {type:"integer"}
tblend_on = False #@param {type:"boolean"}
#tmix_on = True #@param {type:"boolean"}
tblend_framestep_value = "3" #@param {type:"string"}
#@markdown Set `deflicker_avg_frames` filter size in frames. FFmpeg's default is 5. I haven't tested too much but I would try 100.

#@markdown ---
#@markdown Note: Right now only one visual effect can be enabled at a time (will fix soon). Need to experiment more. `Tblend` works best. Tblend divides your target FPS but blends frames.

#video effects
if enable_visual_effects:
  visual_effects = '-vf'
  #if minterpolate_on:
  #  visual_effects = visual_effects + ' minterpolate=fps=' + str(minterpolate_fps) + ':mi_mode=blend'
  if deflicker_on:
    visual_effects = visual_effects + ' deflicker=s=' + str(deflicker_avg_frames) + ':m=am'
  #if mpdecimate_on:
    #visual_effects = visual_effects + ' mpdecimate'
  if tblend_on:
    visual_effects = visual_effects + ' tblend=all_mode=average,framestep=' + tblend_framestep_value
#  if tmix_on:
#    visual_effects = visual_effects + ' tmix=frames=5:weights=\"1 1 1 1 1\",select=\'not(mod(n\,5))\''
#  if deblock_toggle:
#    x265_params = '-x265-params deblock=-3,-3 '
else:
  visual_effects = ''

if model_path == '/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x2plus.pth':
  upscale_value="2"
else:
  upscale_value="4"

if half_precision_realesrgan:
  upscale_value = upscale_value + " --half"
if face_enhance_gfpgan: 
  upscale_value = upscale_value + " --face_enhance"

###############################################################
init_frame_storage = "/content/frames_storage/init_frame_storage"
esrgan_inference_path = "/content/Real-ESRGAN/"
processed_frame_storage = "/content/frames_storage/upscaled_frame_storage"
###############################################################
%cd /content/
print("Cleaning up from last run...")
!rm -rf "/content/Real-ESRGAN/results"
!rm -rf $init_frame_storage
!rm -rf $processed_frame_storage


if import_mp4:
  anKit.splitFrames(imported_mp4,init_frame_storage) #split mp4 into frames
else:
  anKit.sortFrames(import_frame_folder,init_frame_storage)  #duplicates frames to init_frame_storage
  #not necessary if import_mp4 checked bcuz splitFrames() saves as %d

%cd /content/
anKit.runUpscale(esrgan_inference_path,model_path,upscale_value,init_frame_storage,processed_frame_storage)
output_path_mp4_upscaled = output_path_mp4 + '_upscaled.mp4'
anKit.frames2video(processed_frame_storage,target_fps,output_path_mp4_upscaled)



#END OF UPSCALING PHASE
###############################################################################


%cd /content/Practical-RIFE/

measured_fps = anKit.detect_fps(output_path_mp4_upscaled)
print("Detected average FPS of",output_path_mp4_upscaled,"is",measured_fps)
measured_duration = anKit.detect_duration(output_path_mp4_upscaled)
print("Detected duration INTEGER (in seconds) is",measured_duration)
exp_value = anKit.exp_calc()
print("Warning: Target duration currently rounds to the closest integer.")

output_path_mp4_upscaled_smooth = output_path_mp4_upscaled + '_smooth.mp4'



#anKit.RIFE_frames(target_fps,exp_value,processed_frame_storage,target_scale_RIFE,output_path_mp4_upscaled_smooth)
anKit.RIFE_video(target_fps,exp_value,output_path_mp4_upscaled,target_scale_RIFE,output_path_mp4_upscaled_smooth)
#END OF MOTION SMOOTHING PHASE
#########################################################################

#h265 compression
%cd /content/  
anKit.exportVideo(output_path_mp4_upscaled_smooth,visual_effects,constant_quality,output_path_mp4)

/content
Cleaning up from last run...
Creating/content/frames_storage/init_frame_storage...
/content/drive/MyDrive/pytti_test/images_out/basingse
Copying frames to /content/frames_storage/init_frame_storage for processing...
/content/frames_storage/init_frame_storage
mv: './00237.png' and './00237.png' are the same file
mv: './00054.png' and './00054.png' are the same file
mv: './00062.png' and './00062.png' are the same file
mv: './00107.png' and './00107.png' are the same file
mv: './00041.png' and './00041.png' are the same file
mv: './00016.png' and './00016.png' are the same file
mv: './00064.png' and './00064.png' are the same file
mv: './00235.png' and './00235.png' are the same file
mv: './00173.png' and './00173.png' are the same file
mv: './00277.png' and './00277.png' are the same file
mv: './00310.png' and './00310.png' are the same file
mv: './00243.png' and './00243.png' are the same file
mv: './00012.png' and './00012.png' are the same file
mv: './00175.png' and './00175

In [ ]:
#@markdown #3d Photo Inpainting (For Static Images)

!pip3 install torch==1.4.0+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html
!pip3 install opencv-python==4.2.0.32
!pip3 install vispy==0.6.4
!pip3 install moviepy==1.0.2
!pip3 install transforms3d==0.3.1
!pip3 install networkx==2.3
!sudo apt install sed
%cd /content/
!git clone https://github.com/sadnow/3d-photo-inpainting.git
%cd 3d-photo-inpainting
!sh download.sh
!sed -i 's/offscreen_rendering: True/offscreen_rendering: False/g' argument.yml
%cd image
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
%cd ..
!python main.py --config argument.yml

Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Operation cancelled by user
Reading package lists... Done
Building dependency tree       
Reading state information... Done
sed is already the newest version (4.4-2).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
/content
fatal: destination path '3d-photo-inpainting' already exists and is not an empty directory.
/content/3d-photo-inpainting
mkdir: cannot create directory ‘checkpoints’: File exists
downloading from filebox ...
--2021-09-17 17:56:29--  https://filebox.ece.vt.edu/~jbhuang/project/3DPhoto/model/color-model.pth
Resolving filebox.ece.vt.edu (filebox.ece.vt.edu)... 128.173.88.43
Connecting to filebox.ece.vt.edu (filebox.ece.vt.edu)|128.173.88.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206331633 (197M)
Saving to: ‘color-model.pth’

color-model.pth      21%[===>                ]  42.49M  23.0MB/s               